In [33]:
import networkx as nx
import json
import math
import heapq # priority queue
import datetime

In [34]:
class Data:
    def __init__(self):
        self.data = self.import_data()
        self.graph = self.create_graph(self.data)

    @staticmethod
    def import_data():
        with open('../data/stations.json') as file:
            data = json.load(file)
        return data

    @staticmethod
    def create_graph(data):
        g = nx.Graph()
        for s1 in data['stations']:
            g.add_node(s1['name'], coordinates=s1['coordinates'], line=s1['line'])
            for s2, distance in s1['connected_to'].items():
                 g.add_edge(s1['name'], s2, weight=distance)
        return g

    def get_data(self):
        return self.data
    
    def get_graph(self):
        return self.graph

In [35]:
class Methods:

    LINE_COLORS = {
        1: "#d35590",
        3: "#9e9a3a",
        7: "#df8600",
        9: "#8d5544",
        12: "#b89d4e"
    }

    LINE_INTERVALS = { 
        1: 4, 
        3: 3, 
        7: 4, 
        9: 3, 
        12: 2}

    def __init__(self, data_class):
        self.data = data_class.get_data()

    def get_lines(self, station):
        for s in self.data["stations"]:
            if station == s["name"]:
                lines = s["line"]
                if isinstance(lines, (list, tuple)):
                    return [str(line) for line in lines]
                else:
                    return [str(lines)]
        return None

    def get_line_between(self, station1, station2):
        lines1 = self.get_lines(station1)
        lines2 = self.get_lines(station2)
        common = set(lines1) & set(lines2)
        return next(iter(common), None)

    def get_all_stations(self):
        stations = [s["name"] for s in self.data.get("stations", [])]
        return sorted(stations)

    def get_colors_of_path(self, path):
        lines = []
        for i in range(len(path) - 1):
            station1 = path[i]
            station2 = path[i + 1]
            line = self.get_line_between(station1, station2)
            lines.append(line)
        return [self.LINE_COLORS.get(line) for line in lines]
    
    def get_line_interval(self, line):
        return self.LINE_INTERVALS.get(line)

In [36]:
class Al:
    velocity = 600 # 36 km/h = 600 m/min
    transshipment = 6 # 6 min
    stop_time = 0.5 # 30 s = 0.5 min
    months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    opening_time = {0: 5*60, 1: 5*60, 2: 5*60, 3: 5*60, 4: 5*60, 5: 6*60, 6: 7*60} # in minutes from midnight
    closing_time = 0*60 # 0:00 AM

    @staticmethod
    def h(graph, node1, node2):
        return math.dist(graph.nodes[node1]['coordinates'],graph.nodes[node2]['coordinates'])

    def astar_algorithm(self, graph, start_point, end_point, departure_date, departure_time):
        methods = Methods(Data())
        dd_split = departure_date.split()
        dd_month = dd_split[1]
        dd_split[1] = self.months[dd_split[1]]
        week_day = datetime.date(int(dd_split[2]), dd_split[1], int(dd_split[0])).weekday() # 0=Monday, 6=Sunday
        opening = self.opening_time.get(week_day)
        departure_time = int(departure_time.split(":")[0])*60 + int(departure_time.split(":")[1])

        open_list = [(0, start_point)] # (f, node)
        visited = {}

        g_acc = {station: float('inf') for station in graph.nodes()}
        g_acc[start_point] = max(0, opening - departure_time)

        f_acc = {station: float('inf') for station in graph.nodes()}
        f_acc[start_point] = g_acc[start_point] + self.h(graph, start_point, end_point)

        line_acc = {station: None for station in graph.nodes()}
        line_acc[start_point] = None

        while len(open_list) > 0:
            f, current = heapq.heappop(open_list)

            if current == end_point:
                path = []
                while current in visited:
                    path.append(current)
                    current = visited[current]
                path.append(start_point)
                path.reverse()
                arrival = departure_time + g_acc[end_point]
                arrival_hour = int(arrival // 60)
                arrival_minute = int(arrival % 60)
                arrival_time = "%s, %s %s %s, %02d:%02d" % (self.weekdays[week_day], dd_split[0], dd_month, dd_split[2], arrival_hour, arrival_minute)
                return path, g_acc[end_point], arrival_time

            for n in graph.neighbors(current):
                possible_g = g_acc[current] + graph.get_edge_data(current, n)['weight']/self.velocity + self.stop_time
                
                line_between = methods.get_line_between(current, n)
                if line_acc[current] is not None and line_acc[current] != line_between:
                    possible_g += self.transshipment
                    interval_between = methods.get_line_interval(int(line_between))
                    if interval_between is not None:
                        possible_g += (interval_between - (g_acc[current] % interval_between)) % interval_between

                if possible_g < g_acc[n]:
                    print(current, '->', n, ': g = %.2f' % possible_g)
                    visited[n] = current
                    g_acc[n] = possible_g
                    f_acc[n] = possible_g + self.h(graph, n, end_point)
                    line_acc[n] = line_between
                    heapq.heappush(open_list, (f_acc[n], n))

        return None, None, None

In [37]:
g = Data().get_graph()
path, time, date =  Al().astar_algorithm(g, 'Observatorio', 'Universidad', '24 November 2025', '08:00')
print('Path: %s\nTime: %.2f min\nArrival Date: %s' % (path, time, date))

Observatorio -> Tacubaya : g = 2.60
Tacubaya -> Constituyentes : g = 12.17
Tacubaya -> San Pedro de los Pinos : g = 12.31
Tacubaya -> Patriotismo : g = 11.39
Tacubaya -> Juanacatlán : g = 5.03
Juanacatlán -> Chapultepec : g = 7.15
Chapultepec -> Sevilla : g = 8.49
Sevilla -> Insurgentes : g = 10.06
Insurgentes -> Cuauhtémoc : g = 11.89
Patriotismo -> Chilpancingo : g = 13.48
Cuauhtémoc -> Balderas : g = 13.07
Constituyentes -> Auditorio : g = 15.06
San Pedro de los Pinos -> San Antonio : g = 13.82
Balderas -> Niños Héroes : g = 22.61
Balderas -> Juárez : g = 22.60
Chilpancingo -> Centro Médico : g = 15.90
San Antonio -> Mixcoac : g = 15.63
Auditorio -> Polanco : g = 29.08
Mixcoac -> Barranca del Muerto : g = 18.59
Mixcoac -> Insurgentes Sur : g = 23.59
Centro Médico -> Etiopía : g = 26.37
Centro Médico -> Lázaro Cárdenas : g = 18.16
Centro Médico -> Hospital General : g = 25.59
Niños Héroes -> Hospital General : g = 24.04
Insurgentes Sur -> Hospital 20 de Noviembre : g = 25.29
Hospital